<a href="https://colab.research.google.com/github/ingalegb/ChatB/blob/main/Chatbot_Inteligente_Alura_Sprint1_AG_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Configurar ambiente

In [1]:
#Instalando bibliotecas
import pandas as pd
import re, os, random, pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
!python -m spacy download es_core_news_md
!pip install jellyfish
import jellyfish
!pip install transformers
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import torch

#Definiendo variables del proyecto:
nlp = spacy.load('es_core_news_md')

#Conectando al Google Drive
from google.colab import drive
drive.mount('/content/drive')
folder = '/content/drive/MyDrive/Chatbot'

2023-10-09 13:06:55.865257: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 9.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2. Importar verbos

In [2]:
# Importar la lista_verbos:
lista_verbos = pd.read_pickle('/content/drive/MyDrive/Chatbot/verbos/lista_verbos.pickle')

# Importar el diccionario:
verbos_irregulares = pd.read_pickle('/content/drive/MyDrive/Chatbot/verbos/verbos_irregulares.pickle')


In [3]:
print(lista_verbos)

['parar', 'recomendar', 'cancelar', 'fanatizar', 'amaran o amasen', 'exponer', 'obedecer', 'quejar', 'echar', 'legitimar', 'perjudicar', 'organizar', 'molar', 'objetar', 'considerar', 'golear', 'mover', 'acertar', 'reunir', 'regir', 'ilusionar', 'simpatizar', 'conjeturar', 'helar', 'quitar', 'amariamos', 'destacar', 'llegar', 'sincronizar', 'lesionar', 'seducir', 'asistir', 'conservar', 'acordar', 'salvar', 'relucir', 'graduar', 'forzar', 'dar', 'deplorar', 'batear', 'mofar', 'estropear', 'aplastar', 'wasapeo', 'gestionar', 'suprimir', 'gruñir', 'progresar', 'suscribir', 'noticiar', 'cavar', 'alejar', 'galopar', 'virar', 'medir', 'actualizar', 'humanizar', 'convivir', 'gratificar', 'digerir', 'tocar', 'zonificar', 'amariais', 'aterrizar', 'hojear', 'cometer', 'sufrir', 'reciclar', 'obturar', 'divertir', 'ondear', 'listar', 'determinar', 'alentar', 'sumar', 'reflexionar', 'ames', 'anotar', 'mitificar', 'escribir', 'ilustrar', 'obtener', 'subir', 'socorrer', 'desprender', 'agregar', 'gan

In [4]:
print(verbos_irregulares)


{'soy': 'ser', 'estuviste': 'estar', 'fuiste': 'ir', 'tuviste': 'tener', 'hiciste': 'hacer', 'dijiste': 'decir', 'dimar': 'decir', 'pudiste': 'poder', 'supiste': 'saber', 'pusiste': 'poner', 'viste': 'ver', 'diste': 'dar', 'damar': 'dar', 'viniste': 'venir', 'haya': 'haber', 'cupiste': 'caber', 'valiste': 'valer', 'quisiste': 'querer', 'llegaste': 'llegar', 'contaste': 'contar', 'cuesta': 'costar', 'duraste': 'durar', 'eres': 'ser', 'estas': 'estar', 'vas': 'ir', 'vaya': 'ir', 'tienes': 'tener', 'haces': 'hacer', 'dices': 'decir', 'dime': 'decir', 'puedes': 'poder', 'sabes': 'saber', 'pones': 'poner', 'ves': 'ver', 'das': 'dar', 'dame': 'dar', 'vienes': 'venir', 'has': 'haber', 'cabes': 'caber', 'vales': 'valer', 'quieres': 'querer', 'llegares': 'llegar', 'cuentas': 'contar', 'cuestan': 'costar', 'duro': 'durar', 'seras': 'ser', 'estaras': 'estar', 'iras': 'ir', 'tendras': 'tener', 'haras': 'hacer', 'diras': 'decir', 'digame': 'decir', 'podras': 'poder', 'sabras': 'saber', 'pondras': '

#3. Tratamiento de datos

In [5]:
jellyfish.jaro_winkler_similarity('sentirme','sentir')

0.95

In [6]:
#Función para encontrar la raiz de las palabras
def raiz(palabra):
  radio = 0
  palabra_encontrada = palabra
  for word in lista_verbos:
    confianza = jellyfish.jaro_winkler_similarity(palabra, word)
    if (confianza >= 0.93 and confianza >= radio):
      radio = confianza
      palabra_encontrada = word
  return palabra_encontrada

In [7]:
#Crea una función tratamiento_texto que reciba una frase de texto y devuelva la misma frase pero sin acentuaciones, todo en minúscula y
#sin espacios en blanco adicionales.
def tratamiento_texto(texto):
  texto_sin_acentos =str.maketrans('áéíóú', 'aeiou')
  texto_tratado=texto.lower()
  texto_tratado = texto_tratado.translate(texto_sin_acentos)
  texto_tratado = re.sub(r'[^\w\s]', '',texto_tratado)

  return texto_tratado

In [8]:
texto='¡Hola, cómo estás hoy?'
tratamiento_texto(texto)

'hola como estas hoy'

In [9]:

def reemplazar_terminacion(palabra):
  nueva_palabra = 'tu algoritmo'
  return nueva_palabra.split()[0]


In [10]:
#Función para reemplazar el final de una palabra
#Crea una función reemplazar_terminacion que reciba una palabra e identifique si la misma termina en alguna de las siguientes palabras:
#“es”, “me”, “as”, “te”, “ste”, si coincide, entonces que substituya esa terminación por la letra “r”.
def reemplazar_terminacion(palabra):
    terminaciones = ['ste','es','me','as','te']
    for terminacion in terminaciones:
        if palabra.endswith(terminacion):
          return palabra[:-len(terminacion)] + "r"
    return palabra

palabra_original = 'viniste'
palabra_modificada = reemplazar_terminacion(palabra_original)

print(f"Palabra original: {palabra_original}")
print(f"Palabra modificada: {palabra_modificada}")

Palabra original: viniste
Palabra modificada: vinir


In [11]:
#Función para devolver los tokens normalizados del texto
def normalizar(texto):
  tokens=[]
  doc = nlp(texto)
  for t in doc:
    lemma=verbos_irregulares.get(t.text, t.lemma_.split()[0])
    lemma=re.sub(r'[^\w\s+\-*/]', '', lemma)
    if t.pos_ in ('VERB','PROPN','PRON','NOUN','AUX','SCONJ','ADJ','ADV','NUM') or lemma in lista_verbos:
      if t.pos_=='VERB':
        lemma = reemplazar_terminacion(lemma)
        tokens.append(raiz(tratamiento_texto(lemma)))
      else:
        tokens.append(tratamiento_texto(lemma))

  tokens = list(dict.fromkeys(tokens))
  tokens = list(filter(None, tokens))
  return tokens

In [12]:
normalizar('Yo soy Alejandra y me gusta el deporte')

['yo', 'ser', 'alejandra', 'gustar', 'deporte']

# 4. Cargar bases de documentos

In [13]:
#pregunta →
lista_dialogos = []

#respuesta →
lista_dialogos_respuesta= []

#tipo →
lista_tipo_dialogo =[]

In [14]:
import os
import pandas as pd

directorio = '/content/drive/MyDrive/Chatbot/dialogos'

In [15]:
for archivo in os.listdir(directorio):

  if archivo.endswith('.txt'):

    tipo = archivo.split('.')[0]

    with open(os.path.join(directorio, archivo)) as f:
      lineas = f.readlines()

      for i in range(0, len(lineas), 2):

        pregunta = lineas[i].strip()
        pregunta = re.sub(r'[^\w\s+\-*/]', '', pregunta)
        pregunta = tratamiento_texto(pregunta)
        lista_dialogos.append(pregunta)

        respuesta = lineas[i+1].strip()
        lista_dialogos_respuesta.append(respuesta)

        lista_tipo_dialogo.append(tipo)


In [16]:
datos = {'dialogo': lista_dialogos,
         'respuesta': lista_dialogos_respuesta,
         'tipo': lista_tipo_dialogo,
         'interseccion': pd.Series([0] * len(lista_dialogos), dtype=float),
         'jaro_winkler': pd.Series([0] * len(lista_dialogos), dtype=float),
         'probabilidad': pd.Series([0] * len(lista_dialogos), dtype=float)
         }

df_dialogo = pd.DataFrame(datos)

df_dialogo = df_dialogo.drop_duplicates(keep='first')
df_dialogo.reset_index(drop=True, inplace=True)

df_dialogo.sample(5)

,dialogo,respuesta,tipo,interseccion,jaro_winkler,probabilidad
474,hola como estas llevando el dia,"Bien, gracias. ¿Y tú?",Saludos,0.0,0.0,0.0
882,muy buena edad aun la recuerdo hoy tengo años ...,Felicidades! Es un momento memorable. ¡Disfrut...,Otros,0.0,0.0,0.0
250,que beneficios puedo obtener al usar tus servi...,"Al aprovechar mis servicios, puedes obtener co...",Funcion,0.0,0.0,0.0
265,ahora quien responde,Yo respondo.,Identidad,0.0,0.0,0.0
722,ahora haz una compleja,No quiero.,Otros,0.0,0.0,0.0


# 5. Buscar respuesta del Chatbot

In [17]:
def interseccion(texto1, texto2):
  palabras1 = set(texto1.split())
  palabras2 = set(texto2.split())

  if len(palabras1) == 0:
    return 0
  else:
    interseccion = palabras1 & palabras2
    return len(interseccion) / len(palabras1)
interseccion('hola como vas?','hola como estas mi amigo')

0.6666666666666666

In [18]:
def similarity(texto1, texto2):
  vectorizer = TfidfVectorizer()
  vectors = vectorizer.fit_transform([texto1, texto2])
  return cosine_similarity(vectors)[0][1]
similarity('hola como vas?','hola como estas mi amigo')

0.3563004293331381

In [19]:
def jaro_winkler(texto1, texto2):
  return jellyfish.jaro_winkler_similarity(texto1, texto2)
jaro_winkler('hola como vas?','hola como estas mi amigo')

0.8547619047619047

In [20]:
#Función para verificar si el usuário inició un diálogo
def dialogo(user_response):
  user_response = tratamiento_texto(user_response) #Tratando el texto
  user_response = re.sub(r"[^\w\s]", '', user_response) #Elimina signos de puntuación


  df = df_dialogo.copy()
  vectorizer = TfidfVectorizer()
  dialogos_numero = vectorizer.fit_transform(df_dialogo['dialogo'])
  respuesta_numero = vectorizer.transform([user_response])
  for idx,row in df.iterrows():

    df.at[idx, 'interseccion'] = interseccion(user_response, row['dialogo'])
    df.at[idx, 'similarity'] = similarity(user_response, row['dialogo'])
    df.at[idx, 'jaro_winkler'] = jaro_winkler(user_response, row['dialogo'])
    df.at[idx,'probabilidad'] = max(df.at[idx,'interseccion'],df.at[idx,'similarity'],df.at[idx,'jaro_winkler'])
  df.sort_values(by=['probabilidad','jaro_winkler'], inplace=True, ascending=False)
  probabilidad = df['probabilidad'].head(1).values[0]
  if probabilidad >= 0.92:
    print('Respuesta encontrada por el método de comparación de textos - Probabilidad: ', probabilidad)
    respuesta = df['respuesta'].head(1).values[0]
  else:
    respuesta = ''
  return respuesta

In [21]:
dialogo('hola como estas mi amigo?')

Respuesta encontrada por el método de comparación de textos - Probabilidad:  0.925


'Estoy bien, gracias. ¿Y tú?'

In [22]:
from sklearn.preprocessing import LabelEncoder
#Normalizando las frases
label_encoder = LabelEncoder()
df_dialogo['palabras'] = df_dialogo['dialogo'].apply(lambda x: ' '.join(normalizar(x)))
df_dialogo['tipo_num'] = label_encoder.fit_transform(df_dialogo['tipo'])
df_dialogo = df_dialogo[df_dialogo.palabras.values!='']
df_dialogo

,dialogo,respuesta,tipo,interseccion,jaro_winkler,probabilidad,palabras,tipo_num
0,bueno te elogio a ti,¡Muchas gracias! Me alegro mucho de oír eso.,Agradecimiento,0.0,0.0,0.0,bueno tu elogio,0
1,de acuerdo,¡Gracias!,Agradecimiento,0.0,0.0,0.0,acuerdo,0
2,entiendo,¡Perfecto! ¡Me alegra que hayas entendido!,Agradecimiento,0.0,0.0,0.0,entender,0
3,estoy de acuerdo,¡Gracias!,Agradecimiento,0.0,0.0,0.0,estar acuerdo,0
4,igualmente,¡Igualmente!,Agradecimiento,0.0,0.0,0.0,igualmente,0
...,...,...,...,...,...,...,...,...
1146,bueno me voy a dormir buenas noches,Buenas noches 😊 Deseo que descanses bien.,Despedida,0.0,0.0,0.0,bueno yo ir dormir buena noche,4
1147,bueno ps ya paz y amor,¡Hasta luego! Haz el amor y no la guerra!,Despedida,0.0,0.0,0.0,bueno ps ya paz amor,4
1148,vale,¡Hasta la vista!,Despedida,0.0,0.0,0.0,valer,4
1149,buena noche,¡Buenas noches!,Despedida,0.0,0.0,0.0,buen noche,4


In [23]:
# Imprimir diccionario
relacion_diccionario = {}

# Iterar sobre las filas del DataFrame
for tipo, tipo_num in zip(df_dialogo['tipo'], df_dialogo['tipo_num']):
    relacion_diccionario[tipo_num] = tipo

# Imprimir el diccionario
print(relacion_diccionario)

{0: 'Agradecimiento', 1: 'Aprendizaje', 5: 'Edad', 6: 'ElProfeAlejo', 7: 'Error', 8: 'Funcion', 9: 'Identidad', 10: 'Nombre', 11: 'Origen', 13: 'Saludos', 14: 'Sentimiento', 15: 'Usuario', 12: 'Otros', 2: 'Contacto', 3: 'Continuacion', 4: 'Despedida'}


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

##Naive Bayes (Básico)

In [25]:
X = df_dialogo['palabras']
y = df_dialogo['tipo_num']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

vectorizer = CountVectorizer()
X_train= vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

modelo_nb = MultinomialNB()
modelo_nb.fit(X_train, y_train)

y_pred = modelo_nb.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Precisión:", accuracy)

Precisión: 0.6395348837209303


In [26]:
# Guardar el vectorizador y el modelo en un archivo
with open('modelo_naive_bayes.pickle', 'wb') as archivo:
    pickle.dump((vectorizer, modelo_nb), archivo)

# Cargar el vectorizador y el modelo desde el archivo
with open('modelo_naive_bayes.pickle', 'rb') as archivo:
    vectorizer, modelo_nb = pickle.load(archivo)

In [27]:
# Calcular la precisión por clase
unique_classes = df_dialogo['tipo_num'].unique()
for cls in unique_classes:
    cls_indices = y_test == cls
    cls_accuracy = accuracy_score(y_test[cls_indices], y_pred[cls_indices])
    print("Accuracy para la clase", df_dialogo[df_dialogo.tipo_num == cls]['tipo'].unique()[0], ":", cls_accuracy)

Accuracy para la clase Agradecimiento : 0.6842105263157895
Accuracy para la clase Aprendizaje : 0.4166666666666667
Accuracy para la clase Edad : 0.36363636363636365
Accuracy para la clase ElProfeAlejo : 0.4
Accuracy para la clase Error : 0.3
Accuracy para la clase Funcion : 0.3076923076923077
Accuracy para la clase Identidad : 0.38461538461538464
Accuracy para la clase Nombre : 0.2
Accuracy para la clase Origen : 0.5625
Accuracy para la clase Saludos : 0.8703703703703703
Accuracy para la clase Sentimiento : 0.43478260869565216
Accuracy para la clase Usuario : 0.4166666666666667
Accuracy para la clase Otros : 0.9101123595505618
Accuracy para la clase Contacto : 0.42857142857142855
Accuracy para la clase Continuacion : 0.5454545454545454
Accuracy para la clase Despedida : 0.8461538461538461


In [28]:
# Procesando la nueva frase
frase = ' '.join(normalizar('como haces para aprender tan rapido?'))
nueva_frase_vect = vectorizer.transform([frase])

# Realizar la predicción
prediccion = modelo_nb.predict(nueva_frase_vect)

diccionario = {14: 'Sentimiento', 13: 'Saludos', 10: 'Nombre', 9: 'Identidad', 6: 'ElProfeAlejo', 1: 'Aprendizaje', 8: 'Funcion', 15: 'Usuario', 11: 'Origen', 5: 'Edad', 0: 'Agradecimiento', 3: 'Continuacion', 2: 'Contacto', 4: 'Despedida', 12: 'Otros', 7: 'Error'}
llave_buscada = prediccion[0]
clase_encontrada = diccionario[llave_buscada]

print("La frase", frase, "se clasifica como: ", clase_encontrada)

La frase como hacer aprender tanto rapido se clasifica como:  Otros


##Random Forest (Intermedio)

In [29]:
from sklearn.ensemble import RandomForestClassifier
modelo_rf = RandomForestClassifier(n_estimators = 500, n_jobs=-1,random_state=123)
modelo_rf.fit(X_train, y_train)

y_pred = modelo_rf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Precisión:", accuracy)

Precisión: 0.6947674418604651


In [30]:
#Normalizando las frases
label_encoder = LabelEncoder()
df_dialogo['palabras'] = df_dialogo['dialogo'].apply(lambda x: ' '.join(normalizar(x)))
df_dialogo['tipo_num'] = label_encoder.fit_transform(df_dialogo['tipo'])
df_dialogo = df_dialogo[df_dialogo.palabras.values!='']
df_dialogo

,dialogo,respuesta,tipo,interseccion,jaro_winkler,probabilidad,palabras,tipo_num
0,bueno te elogio a ti,¡Muchas gracias! Me alegro mucho de oír eso.,Agradecimiento,0.0,0.0,0.0,bueno tu elogio,0
1,de acuerdo,¡Gracias!,Agradecimiento,0.0,0.0,0.0,acuerdo,0
2,entiendo,¡Perfecto! ¡Me alegra que hayas entendido!,Agradecimiento,0.0,0.0,0.0,entender,0
3,estoy de acuerdo,¡Gracias!,Agradecimiento,0.0,0.0,0.0,estar acuerdo,0
4,igualmente,¡Igualmente!,Agradecimiento,0.0,0.0,0.0,igualmente,0
...,...,...,...,...,...,...,...,...
1146,bueno me voy a dormir buenas noches,Buenas noches 😊 Deseo que descanses bien.,Despedida,0.0,0.0,0.0,bueno yo ir dormir buena noche,4
1147,bueno ps ya paz y amor,¡Hasta luego! Haz el amor y no la guerra!,Despedida,0.0,0.0,0.0,bueno ps ya paz amor,4
1148,vale,¡Hasta la vista!,Despedida,0.0,0.0,0.0,valer,4
1149,buena noche,¡Buenas noches!,Despedida,0.0,0.0,0.0,buen noche,4


In [31]:
# Imprimir diccionario
relacion_diccionario = {}

# Iterar sobre las filas del DataFrame
for tipo, tipo_num in zip(df_dialogo['tipo'], df_dialogo['tipo_num']):
    relacion_diccionario[tipo_num] = tipo

# Imprimir el diccionario
print(relacion_diccionario)

{0: 'Agradecimiento', 1: 'Aprendizaje', 5: 'Edad', 6: 'ElProfeAlejo', 7: 'Error', 8: 'Funcion', 9: 'Identidad', 10: 'Nombre', 11: 'Origen', 13: 'Saludos', 14: 'Sentimiento', 15: 'Usuario', 12: 'Otros', 2: 'Contacto', 3: 'Continuacion', 4: 'Despedida'}


In [32]:
# Guardar el vectorizador y el modelo en un archivo
with open('modelo_random_forest.pickle', 'wb') as archivo:
    pickle.dump((vectorizer, modelo_rf), archivo)

# Cargar el vectorizador y el modelo desde el archivo
with open('modelo_random_forest.pickle', 'rb') as archivo:
    vectorizer, modelo_rf = pickle.load(archivo)

In [33]:
# Calcular la precisión por clase
unique_classes = df_dialogo['tipo_num'].unique()
for cls in unique_classes:
    cls_indices = y_test == cls
    cls_accuracy = accuracy_score(y_test[cls_indices], y_pred[cls_indices])
    print("Accuracy para la clase", df_dialogo[df_dialogo.tipo_num == cls]['tipo'].unique()[0], ":", cls_accuracy)

Accuracy para la clase Agradecimiento : 0.7894736842105263
Accuracy para la clase Aprendizaje : 0.5
Accuracy para la clase Edad : 0.2727272727272727
Accuracy para la clase ElProfeAlejo : 0.7
Accuracy para la clase Error : 0.8
Accuracy para la clase Funcion : 0.5
Accuracy para la clase Identidad : 0.9230769230769231
Accuracy para la clase Nombre : 0.6
Accuracy para la clase Origen : 0.625
Accuracy para la clase Saludos : 0.8703703703703703
Accuracy para la clase Sentimiento : 0.391304347826087
Accuracy para la clase Usuario : 0.6666666666666666
Accuracy para la clase Otros : 0.7528089887640449
Accuracy para la clase Contacto : 0.42857142857142855
Accuracy para la clase Continuacion : 0.6363636363636364
Accuracy para la clase Despedida : 0.6923076923076923


In [34]:
frase = ' '.join(normalizar('como haces para aprender tan rapido?'))
nueva_frase_vect = vectorizer.transform([frase])

prediccion = modelo_rf.predict(nueva_frase_vect)

diccionario = {14: 'Sentimiento', 13: 'Saludos', 10: 'Nombre', 9: 'Identidad', 6: 'ElProfeAlejo', 1: 'Aprendizaje', 8: 'Funcion', 15: 'Usuario', 11: 'Origen', 5: 'Edad', 0: 'Agradecimiento', 3: 'Continuacion', 2: 'Contacto', 4: 'Despedida', 12: 'Otros', 7: 'Error'}
llave_buscada = prediccion[0]
clase_encontrada = diccionario[llave_buscada]

print("La frase", frase, "se clasifica como: ", clase_encontrada)

La frase como hacer aprender tanto rapido se clasifica como:  Aprendizaje


##Transformers (Avanzado)

In [35]:
!pip install transformers

In [36]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text = 'Esta es una oración de ejemplo.'
tokens = tokenizer(text, return_tensors= 'pt', padding=True, truncation=True)

In [37]:
print(tokens)

{'input_ids': tensor([[  101,  9765,  2050,  9686, 14477,  2030, 21736,  2139,  1041,  6460,
          8737,  4135,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [38]:
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import torch

df_train, df_test = train_test_split(df_dialogo, test_size=0.3, random_state=123)

model_name = 'dccuchile/bert-base-spanish-wwm-uncased'
modelo_trans = BertForSequenceClassification.from_pretrained(model_name, num_labels=df_dialogo['tipo_num'].nunique())
tokenizer = BertTokenizer.from_pretrained(model_name)

train_inputs = tokenizer.batch_encode_plus(
    df_train['palabras'].tolist(),
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

test_inputs = tokenizer.batch_encode_plus(
    df_test['palabras'].tolist(),
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

train_data = torch.utils.data.TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], torch.tensor(df_train['tipo_num'].tolist()))
test_data = torch.utils.data.TensorDataset(test_inputs['input_ids'], test_inputs['attention_mask'], torch.tensor(df_test['tipo_num'].tolist()))

optimizer = torch.optim.AdamW(modelo_trans.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
modelo_trans.to(device)
modelo_trans.train()

train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)

for epoch in range(5):
    total_loss = 0

    for batch in train_dataloader:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        optimizer.zero_grad()

        outputs = modelo_trans(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print("Epoch:", epoch + 1, "Loss:", total_loss)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1 Loss: 122.94479167461395
Epoch: 2 Loss: 98.89569127559662
Epoch: 3 Loss: 79.71316140890121
Epoch: 4 Loss: 59.501348316669464
Epoch: 5 Loss: 44.417615324258804


In [39]:
# Evaluación del modelo
modelo_trans.eval()
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

with torch.no_grad():
    predictions = []
    true_labels = []

    for batch in test_dataloader:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        outputs = modelo_trans(input_ids, attention_mask=attention_mask)

        _, predicted_labels = torch.max(outputs.logits, dim=1)

        predictions.extend(predicted_labels.tolist())
        true_labels.extend(labels.tolist())

accuracy = accuracy_score(true_labels, predictions)
print("Precisión:", accuracy)

Precisión: 0.7122093023255814


In [40]:
# Guardar el modelo entrenado
ruta_modelo = '/content/modelo'
modelo_trans.save_pretrained(ruta_modelo)
tokenizer.save_pretrained(ruta_modelo)

('/content/modelo/tokenizer_config.json',
 '/content/modelo/special_tokens_map.json',
 '/content/modelo/vocab.txt',
 '/content/modelo/added_tokens.json')

In [41]:
#Cargar el modelo entrenado
ruta_modelo = '/content/modelo'
modelo_tf = BertForSequenceClassification.from_pretrained(ruta_modelo)
tokenizer_tf = BertTokenizer.from_pretrained(ruta_modelo)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [42]:
# Calcular la precisión por clase
unique_classes = df_dialogo['tipo_num'].unique()

for class_label in unique_classes:
    # Filtrar los datos por clase
    class_data = df_dialogo[df_dialogo['tipo_num'] == class_label]

    # Preparar los datos de la clase para evaluar
    tokens = tokenizer_tf.batch_encode_plus(
        class_data['palabras'].tolist(),
        truncation=True,
        padding=True,
        return_tensors='pt'
    )

    inputs = tokens['input_ids']
    attention_mask = tokens['attention_mask']
    labels = class_data['tipo_num'].tolist()

    # Pasar los datos de la clase por el modelo
    with torch.no_grad():
        outputs = modelo_tf(inputs, attention_mask=attention_mask)

    predicted_labels = outputs.logits.argmax(dim=1).tolist()

    # Calcular la precisión para la clase
    accuracy = accuracy_score(labels, predicted_labels)
    print(f"Precisión por clase {df_dialogo[df_dialogo.tipo_num == class_label]['tipo'].unique()[0]}: {accuracy}")

Precisión por clase Agradecimiento: 0.9393939393939394
Precisión por clase Aprendizaje: 0.7906976744186046
Precisión por clase Edad: 0.3870967741935484
Precisión por clase ElProfeAlejo: 0.8518518518518519
Precisión por clase Error: 0.7083333333333334
Precisión por clase Funcion: 0.7808219178082192
Precisión por clase Identidad: 0.9135802469135802
Precisión por clase Nombre: 0.7916666666666666
Precisión por clase Origen: 0.92
Precisión por clase Saludos: 0.9555555555555556
Precisión por clase Sentimiento: 0.8591549295774648
Precisión por clase Usuario: 0.6486486486486487
Precisión por clase Otros: 0.9104938271604939
Precisión por clase Contacto: 0.6551724137931034
Precisión por clase Continuacion: 0.8064516129032258
Precisión por clase Despedida: 0.9444444444444444


In [43]:
frase = ' '.join(normalizar('hola como estas mi amigo?'))

tokens = tokenizer_tf.encode_plus(
    frase,
    add_special_tokens=True,
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

input_ids = tokens['input_ids']
attention_mask = tokens['attention_mask']

with torch.no_grad():
    outputs = modelo_tf(input_ids, attention_mask)


etiquetas_predichas = torch.argmax(outputs.logits, dim=1)

etiquetas_decodificadas = etiquetas_predichas.tolist()

diccionario = {14: 'Sentimiento', 13: 'Saludos', 10: 'Nombre', 9: 'Identidad', 6: 'ElProfeAlejo', 1: 'Aprendizaje', 8: 'Funcion', 15: 'Usuario', 11: 'Origen', 5: 'Edad', 0: 'Agradecimiento', 3: 'Continuacion', 2: 'Contacto', 4: 'Despedida', 12: 'Otros', 7: 'Error'}
llave_buscada = etiquetas_decodificadas[0]
clase_encontrada = diccionario[llave_buscada]
print("La frase", frase, "se clasifica como: ", clase_encontrada)

La frase hola como estar amigo se clasifica como:  Saludos


In [44]:
#Función para dialogar utilizando el modelo
def clasificacion_modelo(pregunta):
  frase = ' '.join(normalizar(pregunta))
  clase_encontrada = 'Realiza la predicción de la frase con tu modelo aqui'

  #Buscar respuesta más parecida en la clase encontrada
  df = df_dialogo[df_dialogo['tipo'] == clase_encontrada]
  df.reset_index(inplace=True)
  vectorizer = TfidfVectorizer()
  dialogos_num = vectorizer.fit_transform(df['dialogo'])
  pregunta_num = vectorizer.transform([tratamiento_texto(pregunta)])
  similarity_scores = cosine_similarity(dialogos_num, pregunta_num)
  indice_pregunta_proxima = similarity_scores.argmax()

  if max(similarity_scores)>0.5 and clase_encontrada not in ['Otros']:
    print('Respuesta encontrada por el modelo Transformers - tipo:',clase_encontrada)
    respuesta = df['respuesta'][indice_pregunta_proxima]
  else:
    respuesta = ''
  return respuesta


In [45]:
#Función para devolver una respuesta final buscada en todos los métodos disponibles
def respuesta_chatbot(pregunta):
  respuesta = dialogo(pregunta)
  if respuesta != '':
    return respuesta
  else:
    respuesta = clasificacion_modelo(pregunta)
    if respuesta != '':
      return respuesta
    else:
      return 'Respuesta no encontrada'

# 6. Ejecutar Chatbot

In [47]:
pregunta='hola como estas mi hermano'
respuesta = respuesta_chatbot(pregunta)
print(respuesta)

Respuesta encontrada por el método de comparación de textos - Probabilidad:  0.9279352226720647
Estoy bien, gracias por preguntar. ¿Y tú?
